In [24]:
# import functions
import pandas as pd
import numpy as np
import codecs
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler                 #normalising features
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.pipeline import make_pipeline
from sklearn.cross_validation import KFold

from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.svm import SVC, SVR

from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor

from scipy.optimize import minimize

import theano
import theano.tensor as T

%matplotlib inline

np.random.seed=10


In [8]:
# Generating population
% time
num_pop = 100     #number of population
num_date = 100    #number of days to be simulated
num_time = 24     #time intervals; 24 for hourly data, 96 for 15min data; 1440 for 1min data

hist_data = []
# pd.DataFrame(columns=['personID', 'income', 'employment', 'children', 'Date', 'time', 'zone'])


for npop in range(num_pop):
    income = np.random.normal(loc=25000, scale=5000)
    employment = np.random.randint(0, 5)  #assuming 4 types of employment
    children = np.random.randint(0, 2)    #have children or no children
    for ndate in range(num_date):
        for ntime in range(num_time):
            zone = np.random.randint(0,50)  #number of zones, assuming 50
            hist_data.append([npop, income, employment, children, ndate, ntime, zone])
        
hist_data = pd.DataFrame(hist_data, columns=['personID', 'income', 'employment', 'children', 'date', 'time', 'zone'])


Wall time: 0 ns


In [9]:
# making predictions

X = hist_data.drop('zone', axis=1)
y = hist_data['zone']

trainX, testX, trainY, testY = train_test_split(X, y)



In [11]:
## using KNN
# weights might need to be changed - more weights towards the recent data?
# metrics? canberra, minkowski

clf_knn = KNeighborsClassifier(n_neighbors=1).fit(trainX, trainY)
pred_knn = clf_knn.predict_proba(testX)[:, 1]
print precision_score(testY, pred_knn, average='micro')  
print recall_score(testY, pred_knn, average='micro')
print 1./50


0.0192666666667
0.0192666666667
0.02


In [102]:
## using RF

clf_rf = RandomForestClassifier(n_estimators=100, max_depth=30)
clf_rf.fit(trainX, trainY)
pred_rf = clf_rf.predict(testX)
print precision_score(testY, pred_rf, average='micro')  
print recall_score(testY, pred_rf, average='micro')
print 1./50

0.0205666666667
0.0205666666667
0.02


In [101]:
## using GBDT

clf_gb = GradientBoostingClassifier(n_estimators=100, max_depth=3)
clf_gb.fit(trainX, trainY)
pred_gb = clf_gb.predict(testX)
print precision_score(testY, pred_gb, average='micro')  
print recall_score(testY, pred_gb, average='micro')
print 1./50


0.0216833333333
0.0216833333333
0.02


In [20]:
# neural network
def fit_predict_NN(trainX, trainY, testX, activate_functions, hidden_layers):
    X_ = theano.shared(trainX, name='X')
    y_ = theano.shared(trainY, name='y')
    param = T.vector()
    dim = X.shape[1]
    
    def activation(data_, parameter):
        n_previous = 0
        dim_previous = dim
        h = data_
        for n_hidden, func in zip(hidden_layers, activate_functions):
            N = dim_previous * n_hidden
            W_ = parameter[n_previous:n_previous + N].reshape((dim_previous, n_hidden))
            h = func(h.dot(W_))
            dim_previous = n_hidden
            n_previous += N

        # output     
        v_ = parameter[n_previous:]
        output = h.dot(v_)
        n_previous = n_previous + dim_previous
        
        return T.nnet.sigmoid(output), n_previous

    p_sig = activation(X_, param)[0]
    p_bck = 1 - p_sig
    llh_ = y_.dot(T.log(p_sig)) + (1 - y_).dot(T.log(p_bck))
    loss = -llh_
    
    # optimize
    loss_function = theano.function([param], loss)
    loss_grad = theano.function([param], theano.grad(loss, param))
    result = minimize(loss_function, jac=loss_grad, x0=numpy.random.normal(size=activation(X_, param)[1]))
    optimal_params = result['x']
    
    # predict data
    data = T.matrix()
    compiled_activation = theano.function([data, param], activation(data, param)[0])
    
    return compiled_activation(testX, optimal_params)


In [21]:
## define activation function
#sigmoid (which we used, T.nnet.sigmoid)
#leaky ReLU (defined below)
#softplus (T.nnet.softplus)

def LeakyReLU(x):
    return T.switch(x > 0, x, 0.5 * x)


In [26]:
np.random.randint(0, 5, size=1)

array([4])

In [75]:
import sklearn
sklearn.__version__

'0.17'

In [25]:

x = T.vector(name='x') 
function = T.sum(x * x)
function_gradient = T.grad(function, x)

In [26]:
compiled_gradient = theano.function([x], function_gradient)

INFO (theano.gof.compilelock): Refreshing lock C:\Users\Yanjie\AppData\Local\Theano\compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_42_Stepping_7_GenuineIntel-2.7.11-64\lock_dir\lock
INFO:theano.gof.compilelock:Refreshing lock C:\Users\Yanjie\AppData\Local\Theano\compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_42_Stepping_7_GenuineIntel-2.7.11-64\lock_dir\lock
00001	#include <Python.h>
00002	#include <iostream>
00003	#include "theano_mod_helper.h"
00004	#include <math.h>
00005	#include <numpy/arrayobject.h>
00006	#include <numpy/arrayscalars.h>
00007	//////////////////////
00008	////  Support Code
00009	//////////////////////
00010	
00011	
00012	    namespace {
00013	    struct __struct_compiled_op_5558d5d588540c868bea4e3d250b251f {
00014	        PyObject* __ERROR;
00015	
00016	        PyObject* storage_V3;
00017	PyObject* storage_V1;
00018	        
00019	
00020	        __struct_compiled_op_5558d5d588540c868bea4e3d250b251f() {
00021	            // This is only somewhat 

In file included from C:\Users\Yanjie\Anaconda\include/Python.h:58:0,
                 from C:\Users\Yanjie\AppData\Local\Theano\compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_42_Stepping_7_GenuineIntel-2.7.11-64\tmputag9d\mod.cpp:1:
C:\Users\Yanjie\Anaconda\include/pyport.h:886:2: error: #error "LONG_BIT definition appears wrong for platform (bad gcc/glibc config?)."
 #error "LONG_BIT definition appears wrong for platform (bad gcc/glibc config?)."
  ^
In file included from C:\Users\Yanjie\Anaconda\include/Python.h:58:0,
                 from C:\Users\Yanjie\AppData\Local\Theano\compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_42_Stepping_7_GenuineIntel-2.7.11-64\tmputag9d\mod.cpp:1:
C:\Users\Yanjie\Anaconda\include/pyport.h:171:33: error: expected initializer before 'Py_uintptr_t'
 typedef unsigned PY_LONG_LONG   Py_uintptr_t;
                                 ^
In file included from C:\Users\Yanjie\Anaconda\include/Python.h:8:0,
                 from C:\Users\Yanjie\Ap

00001	#include <Python.h>
00002	#include <iostream>
00003	#include "theano_mod_helper.h"
00004	#include <math.h>
00005	#include <numpy/arrayobject.h>
00006	#include <numpy/arrayscalars.h>
00007	#include <vector>
00008	#include <algorithm>
00009	//////////////////////
00010	////  Support Code
00011	//////////////////////
00012	
00013	
00014	    namespace {
00015	    struct __struct_compiled_op_e591ca2cdb54a38a86447e377b272855 {
00016	        PyObject* __ERROR;
00017	
00018	        PyObject* storage_V3;
00019	PyObject* storage_V1;
00020	        
00021	
00022	        __struct_compiled_op_e591ca2cdb54a38a86447e377b272855() {
00023	            // This is only somewhat safe because we:
00024	            //  1) Are not a virtual class
00025	            //  2) Do not use any virtual classes in the members
00026	            //  3) Deal with mostly POD and pointers
00027	
00028	            // If this changes, we would have to revise this, but for
00029	            // now I am tired of chasing se

Exception: ('The following error happened while compiling the node', Elemwise{add,no_inplace}(x, x), '\n', 'Compilation failed (return status=1): In file included from C:\\Users\\Yanjie\\Anaconda\\include/Python.h:58:0,.                  from C:\\Users\\Yanjie\\AppData\\Local\\Theano\\compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_42_Stepping_7_GenuineIntel-2.7.11-64\\tmpwust97\\mod.cpp:1:. C:\\Users\\Yanjie\\Anaconda\\include/pyport.h:886:2: error: #error "LONG_BIT definition appears wrong for platform (bad gcc/glibc config?).".  #error "LONG_BIT definition appears wrong for platform (bad gcc/glibc config?).".   ^. In file included from C:\\Users\\Yanjie\\Anaconda\\include/Python.h:58:0,.                  from C:\\Users\\Yanjie\\AppData\\Local\\Theano\\compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_42_Stepping_7_GenuineIntel-2.7.11-64\\tmpwust97\\mod.cpp:1:. C:\\Users\\Yanjie\\Anaconda\\include/pyport.h:171:33: error: expected initializer before \'Py_uintptr_t\'.  ty